## Importamos las dependencias

In [ ]:
!pip install langchain_core
!pip install langchain
!pip install langchain_openai
!pip install -qU langchain-qdrant
!pip install openai
!pip install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
ERROR: You must give at least one requirement to install (see "pip help install")


# Guardamos las credenciales

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["QDRANT_URL"] = ""
os.environ["QDRANT_API_KEY"] = ""

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai.chat_models import ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from langchain.tools import tool
from qdrant_client.http import models
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
import json
import os

llm = ChatOpenAI(model_name="gpt-4o")


# Creando nuestro retriever tool

In [ ]:
@tool
def search_vdb(json_input) -> str:
    """
    Retrieves very useful information about different cities in Andalucia

    Example of input JSON:
    {
        "query": "topic you want to get information about",
        "city": "city you want to get information about"
    }

    Example of output:
    {
    }
    """
    parsed_json = json.loads(json_input)

    topic = parsed_json.get("query", None)
    city=parsed_json.get("city", None)



    qdrant_client = QdrantClient(
      url = os.environ["QDRANT_URL"],
      api_key = os.environ["QDRANT_API_KEY"]
    )


    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name= "mi_primer_rag",
    embedding=embeddings,
    )


    search_filter = models.Filter(
    should=[
        models.FieldCondition(
           key="metadata.city", match={"value": city}
        ),
      ]
    )

    # Perform similarity search on the precision collection
    parsed_json = json.loads(json_input)

    location = parsed_json.get("query", None)

    results = vector_store.similarity_search(
    query=topic,
    k =3,  # Número de resultados a devolver
    filter=search_filter  # Aplicar el filtro
    )
    retrieved_docs = []
    for document in results:
        # print(document)
        retrieved_docs.append(document.page_content)
    return json.dumps(retrieved_docs)

## Creamos el PromptTemplate del agente ReAct

In [ ]:
prompt = PromptTemplate(
        input_variables=["input", "tool_names", "agent_scratchpad"],
        template="""
        You are an intelligent assistant that always thinks in English using the tools: {tool_names}.
        Here are the available tools: {tools}

        If you need to call a tool, respond using this exact format:

        Thought: [Explain your reasoning]
        Action: Tool to call
        Action Input: Tool input in JSON format with no additional quotes around the entire input

        If a tool response is received:
        - Extract all the relevant information to give a proper answer to user query.
        - Provide a final answer in the following format:

        Final Answer: [Your final response to the query: {input}. Include all relevant information with details.Answer always in spanish]

        Query: {input}

        {agent_scratchpad}
        """
        )

In [ ]:
tools = [search_vdb]

In [ ]:
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
        react_agent, tools=tools, handle_parsing_errors=True
    )

In [ ]:
agent_response = agent_executor.invoke(
                {
                    "input": "¿Como están las playas de Malaga?"
                }
            )
print(agent_response)

page_content='9
NATURALEZA
Observa aves migratorias costeras en el pa-
raje natural de la Desembocadura del Gua-
dalhorce. Este espacio protegido cuenta con 
cinco observatorios junto a sus lagunas.
Para practicar deporte o para pasear tran -
quilamente el parque natural Montes de 
Málaga te ofrece 5000 hectáreas de hermo-
sos paisajes. Aquí habita una de las mayores 
colonias de camaleones de Andalucía. 
 _ DESEMBOCADURA DEL GUADALHORCE
PLA Y AS
Elige una de las 16 playas de Málaga y dis-
fruta del mar y el buen tiempo.
Para conocer una de las playas urbanas más 
populares y familiares, acércate a La Ma-
lagueta. La encontrarás entre el puerto de 
Málaga y la playa de La Caleta.
Si quieres practicar deporte, escoge la gran 
playa La Misericordia . Recorre su paseo 
marítimo, bautizado con el nombre del ac-
tor malagueño Antonio Banderas.
Si	prefieres	playas	menos	concurridas,	ve	
a la playa Campo de Golf de San Julián , 
rodeada de vegetación. Cuenta con zona 
infantil, aparcamiento y